In this notebook we define the BZ map and its associated constants.

The BZ map was introduced in [MT], it is a $C^1$ map on $[0, 1]$
piecewise defined.

We need to compute some constants to guarantee continuity of the derivative and some dynamical properties.

On $[0, \frac{1}{8}]$ the map is defined by 
$$
T_{a,b}(x)=\left(a+\left(\frac{1}{8}-x\right)^{\frac{1}{3}} \right)e^{-x}+b
$$

On $[\frac{1}{8}, 0.3]$ the map is defined by
$$
T_{a,b}(x)=\left(a+\left(x-\frac{1}{8}\right)^{\frac{1}{3}} \right)e^{-x}+b
$$

On $[0.3, 1]$ it is defined by 
$$
T_{b,c}(x)= c\left(10x e^{\frac{-10x}{3}}\right)^{19}+b.
$$

We first compute $a, c$ such that $T$ and $T'$ are continuous
in $0.3$.

In [11]:
using Symbolics
@variables a, b, c, x

expr_T_l = (a+(x-1//8)^(1//3))*exp(-x)+b

b + (a + (x - (1//8))^(1//3))*exp(-x)

In [14]:
expr_T_r = c*(BigInt(10)*x*exp(-10//3*x))^(19)+b

b + (10000000000000000000//1)*c*(x^19)*(exp((-10//3)*x)^19)

In [23]:
val_l = substitute(expr_T_l, (Dict(x=> 3//10)))

b + 0.41437257578452685 + 0.7408182206817179a

In [24]:
val_r = substitute(expr_T_r, (Dict(x=> BigInt(3)//BigInt(10))))

b + 6.511914406794438438226970607009915153990943146993446788572238848363428369686777c

In [26]:
eq_val = Equation(val_r, val_l)

b + 6.511914406794438438226970607009915153990943146993446788572238848363428369686777c ~ 0.41437257578452685 + b + 0.7408182206817179a

In [42]:
D = Differential(x)

der_l = expand_derivatives(D(expr_T_l))

(1//3)*((x - (1//8))^(-2//3))*exp(-x) - (a + (x - (1//8))^(1//3))*exp(-x)

In [41]:
der_val_l = substitute(der_l, (Dict(x=> BigInt(3)//BigInt(10))))

0.78928109673243203817846737678116064900183384367454740171389474716012879141371 + -0.4143725757845268601359959883623735943487139086638386408239274178252921207587769 - 0.7408182206817178660668737793178168721822512319990063482953100668080982468773553a

In [39]:
der_r = expand_derivatives(D(expr_T_r))

(190000000000000000000//1)*c*(x^18)*(exp((-10//3)*x)^19) - (1900000000000000000000//3)*c*(x^19)*(exp((-10//3)*x)^19)

In [40]:
der_val_r = substitute(der_r, (Dict(x=> BigInt(3)//BigInt(10))))

0

In [33]:
z = Symbolics.solve_for(der_eq, a)

0.5060735690368222645574810331055643267537768086550220088252712539778929548760316

In [35]:
typeof(z)

Num

In [47]:
der_l_func = Symbolics.build_function(der_l, a, x, expression = Val(false))

RuntimeGeneratedFunction(#=in Symbolics=#, #=using Symbolics=#, :((a, x)->begin
          #= /home/isaia/.julia/packages/SymbolicUtils/qulQp/src/code.jl:349 =#
          #= /home/isaia/.julia/packages/SymbolicUtils/qulQp/src/code.jl:350 =#
          #= /home/isaia/.julia/packages/SymbolicUtils/qulQp/src/code.jl:351 =#
          (+)((*)((*)(-1, (+)(a, (^)((+)(-1//8, x), 1//3))), (exp)((*)(-1, x))), (*)((*)(1//3, (^)((inv)((+)(-1//8, x)), 2//3)), (exp)((*)(-1, x))))
      end))

In [55]:
using IntervalArithmetic

val_der_l = der_l_func(a, Interval{BigFloat}(3)/10)
val_der_func(a) = der_l_func(a, Interval{BigFloat}(3)/10)

val_der_func (generic function with 1 method)

In [58]:
using DualNumbers
val_der_der_a(a) = val_der_func(Dual(a, 1)).epsilon

val_der_der_a (generic function with 1 method)

In [59]:
function root(f, f′, x, ϵ; max_iter = 100)
	for i in 1:max_iter
		x_old = x
		x_mid = Interval(mid(x))
		x = intersect(x, x_mid - f′(x) \ f(x_mid))
		if x_old == x || isempty(x) || diam(x) < ϵ
			return x
		end
	end
	@info "Maximum iterates reached" max_iter, x, f(x)
	return x
end

root (generic function with 1 method)

In [65]:
setprecision(BigFloat, 1024)
A = root(val_der_func, val_der_der_a, Interval{BigFloat}(0,1), 10^(-30))

[0.506073, 0.506074]₁₀₂₄

In [67]:
A, diam(A)

([0.506073, 0.506074]₁₀₂₄, 1.056910082790920656967860540732891930504127590719607619512830523021422106536611143030784500530942734767931986372242010227447406226737893551706605883054416440037119546927701106977814820035227227134162743397297689273748978244133159042912625500642233310485081028833037802056665806194815529895921683031225989500715e-307)

In [69]:
sub_eq_val_A = substitute(eq_val, Dict(a=>A))

b + 6.511914406794438438226970607009915153990943146993446788572238848363428369686776726593025541176182495825888230805826962020754768304662501317099474517672789450016080360306263532818763543873136383969162510850081236786490723744691422325558960437774658203125c ~ [0.789281, 0.789282]₁₀₂₄ + b

In [73]:
z = Symbolics.solve_for(sub_eq_val_A, c)

[0.121205, 0.121206]₁₀₂₄

In [74]:
C = Symbolics.value(z)

[0.121205, 0.121206]₁₀₂₄

In [76]:
C, diam(C)

([0.121205, 0.121206]₁₀₂₄, 1.390671161567000864431395448332752540137009987788957394095829635554502771758698872408926974382819387852542087331897381878220271350970912568035007740861074263206736245957501456549756342151614772544950978154339064833880234531754156635411349342950306987480369774780312897442981323940546749863054846093718407237782e-308)

We have now found the value of $A$ and $C$